In [43]:
RES_DIR="../proj/simulations/results/"
CONFIG_NAME=RES_DIR+"NonFrequentUpdates-#0"

In [44]:
import subprocess
cmd = "opp_scavetool x " + CONFIG_NAME + ".{0} -o NFU_{0}.csv"
export_types = ("sca", "vec")
for t in export_types:
    print(cmd.format(t))
    subprocess.Popen(cmd.format(t).split())

opp_scavetool x ../proj/simulations/results/NonFrequentUpdates-#0.sca -o NFU_sca.csv
opp_scavetool x ../proj/simulations/results/NonFrequentUpdates-#0.vec -o NFU_vec.csv


Exported 935 scalars, 2625 parameters, 2 statistics, 70 histograms
Exported 3112 vectors


## Metrics
Bellow are the metrics that we want to choose the results:
+ link layer throughput => eth mac txPk
+ application layer throughput - => Load in routers (incomingDataRate)
+ end to end delay? => Useful?
+ request-response/communication latency
+ link utilization => Use throughput and channel capacity TODO
+ number of train updates that the server has received by simulation time=> Vector
+ Number of train updates sent as a response to each client request. => Histogram + vector
+ Number of train updates that were discarded per request as a result of being expired => Vector + hist


In [68]:
import pandas as pd

# Vectors - parse vec dataset
linkLayerThroughput = lambda x: (x["name"] == "txPk:vector(packetBytes)") & (x["type"] == "vector")
appLayerThroughput = lambda x: (x["name"].str.contains("DataRate")) & (x["type"] == "vector") #  (x["module"].str.contains("Router")) => Use this to filter only router
clientResponseDelay = lambda x: x["name"] == "timeToResponse"
serverSentTrainUpdates = lambda x: (x["name"] == "serverSentTrainUpdates")
serverDroppedTrainUpdates = lambda x: (x["name"] == "serverDroppedTrainUpdates")
serverReceivedTrainUpdates = lambda x: (x["name"] == "serverReceivedTrainUpdates")
vec_metrics = (linkLayerThroughput, appLayerThroughput, clientResponseDelay, serverSentTrainUpdates, serverDroppedTrainUpdates, serverReceivedTrainUpdates)

# Histograms - parse sca dataset
clientEndToEndDelay = lambda x: (x["name"] == "endToEndDelay:histogram")  & (x["module"].str.contains("client")) & (x["type"] == "histogram")
#sca_metrics = (clientEndToEndDelay, trainEndToEndDelay)
sca_metrics = [clientEndToEndDelay]

# TODO Calculate link use per time
sca = pd.read_csv("NFU_sca.csv")
vec = pd.read_csv("NFU_vec.csv")
# vec[linkLayerThroughput]
#counts = sca["name"].value_counts()
#counts.to_csv('name.csv',index=True)


In [86]:
# Filter original data
def select_f(x, y):
    res = False
    for cond in y:
        res = res | (cond(x))
    return x[res]

select_vec = select_f(vec, vec_metrics)
select_sca = select_f(sca, sca_metrics)

In [87]:
if "type" in select_vec:
    select_vec = select_vec.drop(["type", "attrname", "attrvalue"], axis=1)
if "type" in select_sca:
    select_sca = select_sca.drop(["type", "attrname", "attrvalue", "value", "underflows", "overflows"], axis=1)
select_vec.head(), select_sca.head()

(                                             run  \
 179  NonFrequentUpdates-0-20221208-21:22:50-7525   
 197  NonFrequentUpdates-0-20221208-21:22:50-7525   
 222  NonFrequentUpdates-0-20221208-21:22:50-7525   
 257  NonFrequentUpdates-0-20221208-21:22:50-7525   
 275  NonFrequentUpdates-0-20221208-21:22:50-7525   
 
                                        module                      name  \
 179     MuenchenNetwork.train[1].eth[0].queue   incomingDataRate:vector   
 197     MuenchenNetwork.train[1].eth[0].queue   outgoingDataRate:vector   
 222       MuenchenNetwork.train[1].eth[0].mac  txPk:vector(packetBytes)   
 257  MuenchenNetwork.trainRouter.eth[1].queue   incomingDataRate:vector   
 275  MuenchenNetwork.trainRouter.eth[1].queue   outgoingDataRate:vector   
 
                                                vectime  \
 179  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1 1.1 1.2 ...   
 197  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1 1.1 1.2 ...   
 222  6.307255452421 6.307275232421 16.060277